In [226]:
import pandas as pd
import numpy as np

In [227]:
df1 = pd.read_csv('subtaskB_train.csv').drop(columns=['Id','topic']).rename(columns={'conspiracy': 'Annotator_1'})
df2 = pd.read_csv('subtaskB_train.csv').drop(columns=['Id','topic']).rename(columns={'conspiracy': 'Annotator_2'})
df3 = pd.read_csv('subtaskB_train.csv').drop(columns=['Id','topic']).rename(columns={'conspiracy': 'Annotator_3'})

In [228]:
df1.head()

,comment_text,Annotator_1
0,"Siamo davanti ad una prova, e non solo di quoz...",1
1,La storia dei 2 bimbi di Bergamo - vaccini: qu...,0
2,L'avete capito che non toglieranno il green pa...,0
3,Quindi la farsa dello spazio e della terra a ...,3
4,"In breve tempo, per accedere a Internet, sarà ...",1


In [229]:
df = df1.copy()
df['Annotator_2'] = df2['Annotator_2']
df['Annotator_3'] = df3['Annotator_3']

In [230]:
df.head()

,comment_text,Annotator_1,Annotator_2,Annotator_3
0,"Siamo davanti ad una prova, e non solo di quoz...",1,1,1
1,La storia dei 2 bimbi di Bergamo - vaccini: qu...,0,0,0
2,L'avete capito che non toglieranno il green pa...,0,0,0
3,Quindi la farsa dello spazio e della terra a ...,3,3,3
4,"In breve tempo, per accedere a Internet, sarà ...",1,1,1


In [231]:
df.shape

(810, 4)

In [232]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 810 entries, 0 to 809
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   comment_text  810 non-null    object
 1   Annotator_1   810 non-null    int64 
 2   Annotator_2   810 non-null    int64 
 3   Annotator_3   810 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 25.4+ KB


## random perturbation (per cotrollare che le funzioni funonziano)

In [233]:
# random perturbation (valori legali)
for i in range(500):
  df.loc[np.random.randint(df.shape[0]+1), 'Annotator_1'] = np.random.randint(0,4, dtype=np.int64)
  df.loc[np.random.randint(df.shape[0]+1), 'Annotator_2'] = np.random.randint(0,4, dtype=np.int64)
  df.loc[np.random.randint(df.shape[0]+1), 'Annotator_3'] = np.random.randint(0,4, dtype=np.int64)

In [234]:
df.head()

,comment_text,Annotator_1,Annotator_2,Annotator_3
0,"Siamo davanti ad una prova, e non solo di quoz...",2.0,1.0,1.0
1,La storia dei 2 bimbi di Bergamo - vaccini: qu...,0.0,0.0,1.0
2,L'avete capito che non toglieranno il green pa...,3.0,0.0,1.0
3,Quindi la farsa dello spazio e della terra a ...,3.0,3.0,1.0
4,"In breve tempo, per accedere a Internet, sarà ...",1.0,0.0,1.0


In [235]:
# random perturbation (valori non legali)
for i in range(11):
  df.loc[np.random.randint(df.shape[0]+1), 'Annotator_1'] = np.random.randint(4,5, dtype=np.int64)
  df.loc[np.random.randint(df.shape[0]+1), 'Annotator_2'] = np.random.randint(4,5, dtype=np.int64)
  df.loc[np.random.randint(df.shape[0]+1), 'Annotator_3'] = np.random.randint(4,5, dtype=np.int64)

In [236]:
for col in df.columns[1:]:
  df[col] =pd.to_numeric(df[col], errors='coerce').fillna(0).astype('int64').astype(np.int64)

In [237]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 812 entries, 0 to 811
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   comment_text  810 non-null    object
 1   Annotator_1   812 non-null    int64 
 2   Annotator_2   812 non-null    int64 
 3   Annotator_3   812 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 25.5+ KB


# Cleaning

In [240]:
def check_vals(unique_vals_exp, df, columns, return_dict=False):
  idxs = dict()
  for col in df.columns[1:]:
    unique_vals = np.unique(df[col])
    if not np.array_equal(unique_vals, unique_vals_exp):
      print('Errors found in col', col)
      print('Indexes of errors:', list(df.query(f'{col} not in {unique_vals_exp}').index))
      idxs[col] = list(df.query(f'{col} not in {unique_vals_exp}').index)

  if len(idxs) == 0:
      print('no errors found')

  if return_dict:
    return idxs

In [241]:
unique_vals_exp = [0,1,2,3]
check_vals(unique_vals_exp, df, df.columns[1:])

Errors found in col Annotator_1
Indexes of errors: [30, 57, 225, 481, 501, 525, 562, 594, 630, 731, 769]
Errors found in col Annotator_2
Indexes of errors: [159, 301, 365, 419, 435, 501, 540, 615, 647, 668, 712]
Errors found in col Annotator_3
Indexes of errors: [20, 121, 205, 402, 437, 445, 500, 578, 594, 663, 698]


In [242]:
idx_dict = check_vals(unique_vals_exp, df, df.columns[1:], return_dict=True)
for col, idx in idx_dict.items():
  df.loc[idx, col] = 1

Errors found in col Annotator_1
Indexes of errors: [30, 57, 225, 481, 501, 525, 562, 594, 630, 731, 769]
Errors found in col Annotator_2
Indexes of errors: [159, 301, 365, 419, 435, 501, 540, 615, 647, 668, 712]
Errors found in col Annotator_3
Indexes of errors: [20, 121, 205, 402, 437, 445, 500, 578, 594, 663, 698]


In [243]:
check_vals(unique_vals_exp, df, df.columns[1:])

no errors found


# Gold Standard

In [244]:
df['Gold_Standard'] = 42
for idx ,row in df[['Annotator_1','Annotator_2','Annotator_3']].iterrows():
  vals = np.array([row['Annotator_1'], row['Annotator_2'], row['Annotator_3']])
  if len(np.unique(vals)) == 2:
    df.loc[idx, 'Gold_Standard'] = np.argmax(np.bincount(vals))

to_check = list(df.query('Gold_Standard == 42').index)
print(to_check) # indice records in cui ogni annotatore ha etichettato con etuchette differenti
print(len(to_check))

[2, 7, 12, 15, 16, 22, 23, 24, 25, 26, 27, 30, 31, 32, 33, 35, 37, 41, 42, 43, 44, 45, 47, 48, 50, 52, 53, 60, 66, 71, 73, 75, 77, 78, 80, 83, 86, 90, 92, 95, 98, 100, 101, 102, 103, 105, 110, 112, 114, 115, 116, 121, 126, 127, 130, 133, 137, 140, 142, 145, 146, 150, 151, 152, 153, 155, 159, 160, 163, 164, 165, 167, 169, 172, 174, 175, 178, 183, 188, 189, 193, 195, 197, 200, 204, 208, 210, 213, 214, 217, 219, 222, 230, 231, 234, 237, 239, 240, 242, 243, 246, 248, 249, 250, 252, 260, 263, 264, 265, 266, 269, 272, 273, 275, 276, 277, 278, 279, 280, 282, 283, 284, 285, 288, 289, 293, 295, 297, 298, 299, 300, 303, 304, 307, 308, 309, 310, 311, 313, 314, 315, 317, 318, 319, 322, 324, 325, 328, 330, 333, 334, 335, 345, 346, 348, 352, 353, 354, 359, 366, 367, 369, 371, 372, 375, 380, 381, 385, 387, 388, 391, 392, 394, 396, 398, 399, 401, 402, 407, 408, 409, 410, 413, 420, 421, 423, 424, 425, 427, 431, 432, 433, 434, 436, 437, 438, 440, 441, 442, 445, 446, 447, 449, 450, 451, 453, 456, 457, 45

# Inter-annotator agreement

### **K = P(A) - P(E) / 1 - P(E)**

P(A) = Volte in cui gli annotatori si sono mostrati d'accordo

P(E) = Volte in cui ci si aspetta che gli annotatori per caso compiano la stessa scelta (e = expected)

The higher the better

In [67]:
def interannotator_agreement(annotator1, annotator2):
  num_records = len(annotator1)
  label1, label2, label3, label4 = np.unique(annotator1)

  counts1 = np.unique(annotator1, return_counts=True)[1]
  counts2 = np.unique(annotator2, return_counts=True)[1]

  annotation_prob = {
      'Annotator_1': {
          label1: counts1[0]/num_records,
          label2: counts1[1]/num_records,
          label3: counts1[2]/num_records,
          label4: counts1[3]/num_records
      },
      'Annotator_2': {
          label1: counts2[0]/num_records,
          label2: counts2[1]/num_records,
          label3: counts2[2]/num_records,
          label4: counts2[3]/num_records
      }
  }

  alignment = 0

  for val1, val2 in zip(annotator1, annotator2):
    if val1 == val2:
      alignment += 1

  p_a = alignment/num_records
  p_e = (annotation_prob['Annotator_1'][label1] * annotation_prob['Annotator_2'][label1]) + \
        (annotation_prob['Annotator_1'][label2] * annotation_prob['Annotator_2'][label2]) + \
        (annotation_prob['Annotator_1'][label3] * annotation_prob['Annotator_2'][label3]) + \
        (annotation_prob['Annotator_1'][label4] * annotation_prob['Annotator_2'][label4])

  k =  (p_a - p_e) / (1 - p_e)

  return k

print(interannotator_agreement(np.array([1,1,1,2,2,3,4,4]), np.array([1,1,1,2,2,3,4,4])))



1.0


In [68]:
from statistics import harmonic_mean, mean

k1 = interannotator_agreement(np.array(df['Annotator_1']), np.array(df['Annotator_2']))
k2 = interannotator_agreement(np.array(df['Annotator_1']), np.array(df['Annotator_3']))
k3 = interannotator_agreement(np.array(df['Annotator_2']), np.array(df['Annotator_3']))
agg = (k1+k2+k3)/3

print('k1:', k1)
print('k2:', k2)
print('k3:', k3)
print('Mean:', agg)



k1: 0.983757525930308
k2: 0.9817183726593123
k3: 0.9939009545131681
Mean: 0.9864589510342627
